## Data Preparation

In [1]:
import pandas as pd

## a) cargamos los datos

In [7]:
# indicamos las rutas del dataset
movies_data_path = '../dataset/original_data/movies.csv'
finantial_data_path = '../dataset/original_data/finantials.csv'
opening_data_path = '../dataset/original_data/opening_gross.csv'

In [9]:
# cargamos los datasets en formato de dataframe 
fin_data = pd.read_csv(finantial_data_path)
movie_data = pd.read_csv(movies_data_path)
opening_data = pd.read_csv(opening_data_path)

## b) creamos un solo dataset con todas las columnas y filas utiles 
en nuestro caso tenemos las columnas repartidas en 3 DF, solo couparaemos las columnas numericas asi que elimnamos las categoricas, el numero de registros varia asi que haremos merge para que solo se concerven los registros que existen en todas las tablas. 

In [10]:
# extraemos las columnas que vamos a utilizar (las numericas y la del titulo[sera el key del merge]) del DF movie
"""numeric_columns_mask = (movie_data.dtypes == float) | (movie_data.dtypes == int)
numeric_columns = [column for column in numeric_columns_mask.index if numeric_columns_mask[column]]
movie_data = movie_data[numeric_columns+['movie_title']]"""
movie_data = pd.concat([movie_data.select_dtypes(include=['float64', "int64"]) ,movie_data["movie_title"]], 
       axis=1) # este es mi codigo que cree y es mas facil  axis=1 = unir columnas axis=0 = unir filas

In [11]:
fin_data = fin_data[['movie_title','production_budget','worldwide_gross']] 
# hacemos un left para que se concerben los registros que existen solo en las 3 tablas
fin_movie_data = pd.merge(fin_data, movie_data, on= 'movie_title', how='left') 
full_movie_data = pd.merge( opening_data,fin_movie_data, on = 'movie_title', how='left')
full_movie_data.shape

(2304, 12)

In [12]:
# ignoraremos las peliculas que tienen 0 y null en dinero generado 
full_movie_data[(full_movie_data.worldwide_gross != 0) & (full_movie_data.worldwide_gross.notnull())].shape

(2304, 12)

In [13]:
# quitamos las columnas que no son utiles para el entrenamiento
full_movie_data = full_movie_data.drop(['movie_title','gross'],axis=1)

In [14]:
full_movie_data.columns # vemos el DF final

Index(['opening_gross', 'screens', 'production_budget', 'worldwide_gross',
       'title_year', 'aspect_ratio', 'duration', 'cast_total_facebook_likes',
       'budget', 'imdb_score'],
      dtype='object')

en este punto ya tenemos un solo dataframe con toda la data que sera util para entrenar el modelo 

## Modeling

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate
import numpy as np

In [16]:
# definimos nuestro input var y output var
X = full_movie_data.drop(['worldwide_gross'], axis = 1)
y = full_movie_data['worldwide_gross']

In [17]:
# contruimos un pipeline simple 
pipeline = Pipeline([ # solo especifiamos el alias del modelo y el modelo 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')), # completa los valores Null
    ('core_model', GradientBoostingRegressor()) #modelo de entrenamiento
])

In [18]:
results = cross_validate(pipeline ,X,y,return_train_score=True,cv=10) # cv = numero de bloques 
results # ahora tenemos los datos de puntuacion y tiempo para el fit y el test de las 10 iteraciones 

{'fit_time': array([1.04327774, 0.75268006, 0.74164438, 0.72511768, 0.71393323,
        0.78566194, 0.7423985 , 0.72224951, 0.74199581, 0.74959016]),
 'score_time': array([0.00501299, 0.0091095 , 0.00824523, 0.00600004, 0.00400901,
        0.00984097, 0.0078249 , 0.00499177, 0.00500798, 0.00527096]),
 'test_score': array([0.67415559, 0.8496611 , 0.64348585, 0.77637806, 0.78305169,
        0.85659871, 0.75666105, 0.87453744, 0.67446952, 0.65792353]),
 'train_score': array([0.91673951, 0.91581777, 0.9228721 , 0.91654412, 0.92172829,
        0.91476722, 0.92151444, 0.91734995, 0.92320705, 0.91766026])}

In [19]:
# obtenemos el promedio de las 10 iteraciones del crossvalidation 
train_score = np.mean(results['train_score'])
test_score = np.mean(results['test_score'])
assert train_score > 0.7 # asi se usa assert:
assert test_score > 0.65 #  afirmo que train score es > 0.7 si no deten la ejecucion por que hay un error atras
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')

Train Score: 0.918820072167686
Test Score: 0.7546922555602998


## Hyperparameter tunning

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
# de todos los parametros del modelo GBR definimos uno que sera ajustado y en que rango se haran pruebas
param_tunning = {'core_model__n_estimators': range(20,501,20)} 

In [22]:
# volvemos a instanciar el pipeline de atras
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor())
])

In [23]:
grid_search= GridSearchCV(estimator, # el modelo o pipeline
                        param_grid = param_tunning, # el paramatro a ajustar
                        scoring='r2', # la metrica de validacion 
                        cv=5)         # numero de bloques de particion en el crossvalidation 

In [24]:
# hacemos un hold out 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.35,random_state= 42)

In [25]:
# entrenamos el estimador
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('core_model',
                                        GradientBoostingRegressor())]),
             param_grid={'core_model__n_estimators': range(20, 501, 20)},
             scoring='r2')

In [26]:
# hacemos la tecnica de cross validation sobre el  mejor modelo (estimator) encontrado en grid_search 
final_result = cross_validate(grid_search.best_estimator_,X_train,y_train,return_train_score=True,cv=7)

In [27]:
# volvemos a hacer el promedio de las metricas del crossvalidation para el mejor modelo 
train_score = np.mean(final_result['train_score'])
test_score = np.mean(final_result['test_score'])
assert train_score > 0.7
assert test_score > 0.65
print(f'Train Score: {train_score}')
print(f'Test Score: {test_score}')
# la metrica de evaluacion es r^2

Train Score: 0.9602770500418728
Test Score: 0.7587274917866453


In [28]:
# obtenmso los parametros para el mejor modelo (estimator)
grid_search.best_estimator_.get_params()

{'memory': None,
 'steps': [('imputer', SimpleImputer()),
  ('core_model', GradientBoostingRegressor(n_estimators=180))],
 'verbose': False,
 'imputer': SimpleImputer(),
 'core_model': GradientBoostingRegressor(n_estimators=180),
 'imputer__add_indicator': False,
 'imputer__copy': True,
 'imputer__fill_value': None,
 'imputer__missing_values': nan,
 'imputer__strategy': 'mean',
 'imputer__verbose': 0,
 'core_model__alpha': 0.9,
 'core_model__ccp_alpha': 0.0,
 'core_model__criterion': 'friedman_mse',
 'core_model__init': None,
 'core_model__learning_rate': 0.1,
 'core_model__loss': 'squared_error',
 'core_model__max_depth': 3,
 'core_model__max_features': None,
 'core_model__max_leaf_nodes': None,
 'core_model__min_impurity_decrease': 0.0,
 'core_model__min_samples_leaf': 1,
 'core_model__min_samples_split': 2,
 'core_model__min_weight_fraction_leaf': 0.0,
 'core_model__n_estimators': 180,
 'core_model__n_iter_no_change': None,
 'core_model__random_state': None,
 'core_model__subsample'

In [29]:
# este paso solo es para ver que sucedia en la funcion de save_simple_metrics_report en utils.py
for key, value in grid_search.best_estimator_.named_steps.items():
    print(f'### {key}:{value.__repr__()}'+'\n')

### imputer:SimpleImputer()

### core_model:GradientBoostingRegressor(n_estimators=180)



In [26]:
# creamos el pipeline final con el modelo y sus parametros mas optimos
estimator = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('core_model', GradientBoostingRegressor(n_estimators=220,
                                             alpha=0.9,
                                             ccp_alpha=0.0,
                                             criterion='friedman_mse',
                                             init=None,
                                             learning_rate=0.1,
                                             loss='squared_error',
                                             max_depth=3,
                                             max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_iter_no_change=None,
                                             random_state=None,
                                             subsample=1.0,
                                             tol=0.0001,
                                             validation_fraction=0.1,
                                             verbose=0,
                                             warm_start=False))
])

In [27]:
# entrenamos el modelo final
estimator.fit(X_train,y_train)

Pipeline(steps=[('imputer', SimpleImputer()),
                ('core_model', GradientBoostingRegressor(n_estimators=220))])

In [28]:
estimator.score(X_test, y_test)

0.7297050595008379

## Saving model

In [29]:
from joblib import dump

In [30]:
# guardamos el modelo final ya entrenado en formato .pkl
dump(estimator, '../model/model.pkl')

['../model/model.pkl']

In [31]:
X_train.columns

Index(['opening_gross', 'screens', 'production_budget', 'title_year',
       'aspect_ratio', 'duration', 'cast_total_facebook_likes', 'budget',
       'imdb_score'],
      dtype='object')